
# Sprachmodell-Anfrage mit modernem OpenAI Client und GPT4All-Embeddings

Dieses Jupyter Notebook demonstriert, wie man mit Hilfe des OpenAI-Clients und GPT4All-Embeddings
eine umfassende Textanalyse durchführt. Es verwendet den Hochschulinternen API-Endpunkt.

## Überblick

In diesem Notebook lernen Sie:
1. Wie man die OpenAI-Bibliothek und GPT4All installiert
2. Wie man den modernen OpenAI Client mit benutzerdefinierten Endpunkten konfiguriert
3. Wie man lokale Embeddings mit GPT4All generiert
4. Wie man lange Texte analysiert und mit OpenAI integriert


## 1. Installation der erforderlichen Pakete

Zunächst müssen wir sicherstellen, dass alle notwendigen Bibliotheken installiert sind.

Auf AI.H2.de ist das nicht nötig, da der JupyterHub die Bibliotheken schon vorinstalliert.

In anderen Umgebungen muss das noch nachgeholt werden. Führen Sie die folgende Zelle aus:

In [ ]:
!pip install "openai>=1.0.0" gpt4all langchain langchain_community

## 2. Initialisierung des modernen OpenAI Clients

Wir importieren die `OpenAI`-Klasse und erstellen einen Client mit unseren Konfigurationsparametern.

**Wichtige Parameter:**
- `api_key`: Der Authentifizierungsschlüssel für die API
- `base_url`: Die Basis-URL des API-Servers (ersetzt das frühere `api_base`)

In [ ]:
from openai import OpenAI

# Initialisierung des Clients mit benutzerdefinierten Parametern
client = OpenAI(
    api_key="sk-1234",            # API-Key für die Authentifizierung
    base_url="https://ai.h2.de/llm"  # Benutzerdefinierter Endpunkt
)

print(f"Moderner OpenAI Client initialisiert mit Basis-URL: {client.base_url}")

## 3. Einrichtung des GPT4All-Embeddings-Modells

Wir initialisieren das GPT4All-Embeddings-Modell. Dieses Modell ist besonders geeignet für:
- Lokale Text-Einbettungen
- Effiziente Verarbeitung langer Texte
- Privatsphäre-gerechte Textanalyse

In [ ]:
from gpt4all import Embed4All

# Initialisierung des Embeddings-Modells
embedder = Embed4All()

print("GPT4All-Embeddings-Modell erfolgreich initialisiert")

## 4. Generierung von Embeddings für einen langen Text

In diesem Abschnitt demonstrieren wir, wie man Embeddings für einen langen Text generiert. Das Modell verarbeitet automatisch Texte, die länger als der Kontext sind, indem es sie in Chunks unterteilt und die Embeddings durchschnittlich berechnet.

In [ ]:
# Beispiel für einen langen Text (z.B. ein Artikel oder Dokument)
long_text = """
Die Künstliche Intelligenz (KI) ist eine der wichtigsten Technologien des 21. Jahrhunderts.
Sie revolutioniert Bereiche wie Gesundheitswesen, Finanzwesen, Produktion und Vertrieb.
KI-Systeme können komplexe Muster erkennen, Vorhersagen treffen und selbstständig lernen.
Dabei spielen verschiedene Techniken wie neuronale Netze, maschinelles Lernen und tiefes Lernen
eine zentrale Rolle. Die Entwicklung fortschreitender KI-Modelle wie GPT-4 oder Claude
öffnet neue Möglichkeiten für natürlichsprachliche Interaktionen und intelligente Systeme.
Trotz der Vorteile gibt es auch Herausforderungen wie Datensicherheit, Ethik und
die Notwendigkeit menschlicher Überwachung.
"""

In [ ]:
# Generieren der Embeddings
embeddings = embedder.embed(long_text, long_text_mode="mean")

# Ausgabe der Embedding-Größe
print(f"Generierte Embeddings haben die Dimension: {len(embeddings)}")

## 5. Analyse der Embeddings

Wir können die Embeddings nutzen, um Ähnlichkeiten zwischen Texten zu analysieren. Dafür verwenden wir den Cosinus-Ähnlichkeitsvergleich.

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Beispiel-Anfrage für Ähnlichkeitsvergleich
query = "Welche technischen Grundlagen stehen hinter modernen KI-Systemen?"
query_embedding = embedder.embed(query)

# Cosinus-Ähnlichkeit berechnen
similarity = cosine_similarity([query_embedding], [embeddings])[0][0]
print(f"Ähnlichkeit zwischen Anfrage und Dokument: {similarity:.4f}")

## 6. Integration mit OpenAI für erweiterte Analyse

Wir können die Erkenntnisse aus den Embeddings mit dem OpenAI-Modell kombinieren, um eine detailliertere Analyse durchzuführen.

In [ ]:
# Senden einer Anfrage an das OpenAI-Modell mit den Erkenntnissen aus den Embeddings
analysis_request = f"""
Basierend auf der Textanalyse mit GPT4All-Embeddings:
- Der Text hat eine Embedding-Dimensionalität von {len(embeddings)}
- Die Ähnlichkeit zur Anfrage '{query}' beträgt {similarity:.4f}

Bitte gib eine detaillierte Analyse des Textinhalts und der Fragestellung.
"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",  # Standardmodell für Analyse
    messages=[
        {"role": "system", "content": "Du bist ein hilfreicher Assistent, der bei der Analyse von Text-Embeddings hilft."},
        {"role": "user", "content": analysis_request}
    ],
    temperature=0.3  # Geringe Temperatur für präzise Analyse
)

# Formatierte Ausgabe der Analyse
from IPython.display import Markdown, display

markdown_output = f"""
## Analyse des Textes:

> {completion.choices[0].message.content}

### Metadaten zur Analyse:
- **Modell**: {completion.model}
- **Completion-Tokens**: {completion.usage.completion_tokens}
- **Prompt-Tokens**: {completion.usage.prompt_tokens}
- **Gesamtanzahl der Tokens**: {completion.usage.total_tokens}
"""

display(Markdown(markdown_output))